In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
import numpy as np

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd

In [2]:
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
# need to define following things
# x: observed context
# w in W: decision to be made
# c: parameter of cost/utility function
# f(c, w): cost/utility function

# formulation: min_{w in W} f(c, w) 
# where {c in U(x)}

# (x) 1. generate x
# (x) 2. generate c = g(x)
# (x) 3. choose a function f(c, w)
# (*) 4. determine how to solve min_{w in W} f(c, w) for some simple choices of W: how to solve?
# ( ) 5. produce a dataset {(c, x)}
# ( ) 6. learn a predictor g^(x) = c
# ( ) 7. conformalize g^(x) to produce C(x) regions
# ( ) 8. solve min_{w in C(x)} f(c, w)

# goal: determine allocation of items to buy (where utility of each is unknown)
n = 20 # number of items (c in R^n)
d = 10 # dim of context to utility (x in R^d)

In [12]:
theta = np.random.randint(low=0, high=2, size=(d, n))

def g(x):
    c = (x @ theta) ** 2
    return c

In [13]:
N = 2_000
N_train = int(N * 0.5)

x_dataset = np.random.uniform(low=0, high=4, size=(N, d))
c_dataset = g(x_dataset) * np.random.uniform(low=4/5, high=6/5, size=(N, n))

device = ("cuda" if torch.cuda.is_available() else "cpu")
to_tensor = lambda r : torch.tensor(r).to(torch.float32).to(device)
x_train, x_cal = to_tensor(x_dataset[:N_train]), to_tensor(x_dataset[N_train:])
c_train, c_cal = to_tensor(c_dataset[:N_train]), to_tensor(c_dataset[N_train:])

In [14]:
train_dataset = TensorDataset(x_train, c_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [16]:
model = FeedforwardNN(input_dim=d, output_dim=n).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1_000

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(batch_X)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 10825.3672
Epoch [101/1000], Loss: 1053.7017
Epoch [201/1000], Loss: 368.4078
Epoch [301/1000], Loss: 258.3434
Epoch [401/1000], Loss: 224.2521
Epoch [501/1000], Loss: 188.3053
Epoch [601/1000], Loss: 129.8586
Epoch [701/1000], Loss: 150.2671
Epoch [801/1000], Loss: 143.0212
Epoch [901/1000], Loss: 162.8486


In [20]:
# *marginal* ellipsoid constraint <- sample mean/cov followed by solution
# *conditional* ellipsoid conformal constraint (PTC-E) <- (univariate) Gaussian variational family followed by same as above
# *conditional* generative sampled-based conformal constraint (CSPO)
# *conditional* generative density-based conformal constraint (CDPO)

In [25]:
def box_solve_generic(c_box_lb, c_box_ub, c_true, p, B):
    covered = int(np.all(c_box_lb <= c_true) and np.all(c_true <= c_box_ub))

    # perform RO over constraint region
    model = ro.Model()

    w = model.dvar(n)
    c = model.rvar(n)
    uset = (c_box_lb <= c, c <= c_box_ub)

    model.minmax(-c @ w, uset)
    model.st(w <= 1)
    model.st(w >= 0)
    model.st(p @ w <= B)

    model.solve(grb)
    return covered, model.get()

In [32]:
# *marginal* box constraint (i.e. just ignore contextual information)
def box_solve_marg(alpha, x, c_true, p, B):
    alpha = alpha / 20
    c_box_lb = np.quantile(c_dataset, q=(alpha / 2), axis=0)
    c_box_ub = np.quantile(c_dataset, q=(1 - alpha / 2), axis=0)
    return box_solve_generic(c_box_lb, c_box_ub, c_true, p, B)

In [37]:
# *conditional* box conformal constraint (PTC-B)
def box_solve_cp(alpha, x, c_true, p, B):
    c_pred = model(x_cal)
    box_cal_scores = np.linalg.norm((c_pred - c_cal).detach().cpu().numpy(), np.inf, axis=1)
    conformal_quantile = np.quantile(box_cal_scores, q=1 - alpha, axis=0)

    c_box_hat = model(to_tensor(x)).detach().cpu().numpy()
    c_box_lb = c_box_hat - conformal_quantile
    c_box_ub = c_box_hat + conformal_quantile
    return box_solve_generic(c_box_lb, c_box_ub, c_true, p, B)

In [38]:
def trial(alpha):
    x = np.random.uniform(low=0, high=4, size=(d, 1))[...,0]
    c_true = g(x) * np.random.uniform(low=4/5, high=6/5, size=(1))[...,0]

    p = np.random.randint(low=0, high=1000, size=n)
    u = np.random.uniform(low=0, high=1)
    B = np.random.uniform(np.max(p), np.sum(p) - u * np.max(p))

    marg_box_covered, marg_box_value = box_solve_marg(alpha, x, c_true, p, B)
    cp_box_covered, cp_box_value = box_solve_cp(alpha, x, c_true, p, B)
    return (marg_box_covered, marg_box_value), (cp_box_covered, cp_box_value)

In [35]:
alphas = [0.05]
marg_box_cov_props = []
cp_box_cov_props = []

marg_box_value_means = []
cp_box_value_means = []

n_trials = 100

for alpha in alphas:
    marg_box_covered = 0
    marg_box_values = []

    cp_box_covered = 0
    cp_box_values = []

    for _ in range(n_trials):
        (marg_box_covered_trial, marg_box_value_trial), (cp_box_covered_trial, cp_box_value_trial) = trial(alpha)
        marg_box_covered += marg_box_covered_trial
        marg_box_values.append(marg_box_value_trial)

        cp_box_covered += cp_box_covered_trial
        cp_box_values.append(cp_box_value_trial)

    marg_box_cov_props.append(marg_box_covered / n_trials)
    cp_box_cov_props.append(cp_box_covered / n_trials)

    marg_box_value_means.append(np.mean(marg_box_values))
    cp_box_value_means.append(np.mean(cp_box_values))

Being solved by Gurobi...
Solution status: 2
Running time: 0.0007s
True
Being solved by Gurobi...
Solution status: 2
Running time: 0.0004s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0005s
True
Being solved by Gurobi...
Solution status: 2
Running time: 0.0005s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0005s
True
Being solved by Gurobi...
Solution status: 2
Running time: 0.0005s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0005s
True
Being solved by Gurobi...
Solution status: 2
Running time: 0.0004s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0005s
False
Being solved by Gurobi...
Solution status: 2
Running time: 0.0005s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0005s
True
Being solved by Gurobi...
Solution status: 2
Running time: 0.0004s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0006s
True
Being solved by Gurobi...
Solution status: 2
Running time: 0.0004s
Being solved by Gurobi...


In [36]:
import pandas as pd

coverage_data = {
    r"$\alpha$": alphas,
    "Box": marg_box_cov_props,
    "PTC-B": cp_box_cov_props,
}
coverage_df = pd.DataFrame(coverage_data)

values_data = {
    r"$\alpha$": alphas,
    "Box": marg_box_value_means,
    "PTC-B": cp_box_value_means,
}
values_df = pd.DataFrame(values_data)

In [40]:
values_df

,$\alpha$,Box,PTC-B
0,0.05,-110.541596,-724.424823


In [239]:
print(coverage_df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(coverage_data),
))

print(values_df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(values_data)
))

\begin{tabular}{ccc}
\toprule
$\alpha$ & Box & PTC-B \\
\midrule
0.05 & 0.00 & 0.92 \\
\bottomrule
\end{tabular}

\begin{tabular}{ccc}
\toprule
$\alpha$ & Box & PTC-B \\
\midrule
0.05 & -602.11 & -3.76 \\
\bottomrule
\end{tabular}

